In [2]:
import pandas as pd

factify_train = pd.read_csv("/Users/sakdag/Downloads/public_folder/train.csv")

In [5]:
factify_train["Category"].value_counts()

Category
Support_Text               7000
Insufficient_Multimodal    7000
Support_Multimodal         7000
Insufficient_Text          7000
Refute                     7000
Name: count, dtype: int64

In [6]:
factify_val = pd.read_csv("/Users/sakdag/Downloads/public_folder/val.csv")

In [7]:
factify_val["Category"].value_counts()

Category
Support_Multimodal         1500
Refute                     1500
Support_Text               1500
Insufficient_Text          1500
Insufficient_Multimodal    1500
Name: count, dtype: int64

In [11]:
import pandas as pd
from tqdm import tqdm
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

# Ensuring consistent language detection
DetectorFactory.seed = 0

# Example DataFrame
# factify_train = pd.DataFrame({'claim': ['This is an English sentence.', 'Ceci est une phrase française.', 'Este es un texto en español.', ...]})

def detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        return "unknown"

# Use tqdm to show progress
tqdm.pandas(desc="Detecting languages")

# Apply language detection with progress bar
factify_train['language'] = factify_train['document'].progress_apply(detect_language)

# Get counts per language
language_counts = factify_train['language'].value_counts()

# Display the results
print(language_counts)


Detecting languages: 100%|██████████| 35000/35000 [03:00<00:00, 194.15it/s]

language
en         34969
nl            11
hi             7
de             6
es             2
it             1
fr             1
unknown        1
id             1
bn             1
Name: count, dtype: int64


In [2]:
import pickle
import os

def modify_keys_in_pkl(input_file: str, output_file: str):
    """
    Reads a .pkl file, modifies its keys by replacing backslashes (\\) with slashes (/),
    and writes the updated data back to a new .pkl file.

    Args:
        input_file (str): Path to the input .pkl file.
        output_file (str): Path to the output .pkl file.
    """
    # Ensure the input file exists
    if not os.path.exists(input_file):
        raise FileNotFoundError(f"Input file {input_file} does not exist.")

    # Read the data from the input .pkl file
    with open(input_file, 'rb') as file:
        data = pickle.load(file)

    # Ensure the data is a dictionary
    if not isinstance(data, dict):
        raise ValueError("The .pkl file must contain a dictionary.")

    # Modify the keys in the dictionary
    modified_data = {
        (key.replace('\\', '/') if isinstance(key, str) else key): value
        for key, value in data.items()
    }

    # Write the modified data back to a new .pkl file
    with open(output_file, 'wb') as file:
        pickle.dump(modified_data, file)

    print(f"Modified keys saved to {output_file}")

In [5]:
from src.utils.path_utils import get_project_root

project_root = get_project_root()
old_pkl = os.path.join(project_root, "evidence_feature_old.pkl")
new_pk = os.path.join(project_root, "evidence_feature.pkl")

In [6]:
modify_keys_in_pkl(old_pkl, new_pk)

Modified keys saved to /Users/sakdag/Desktop/masters/multimodal-misinformation-detection/evidence_feature.pkl


In [13]:
with open(new_pk, 'rb') as file:
    data = pickle.load(file)
    print(list(data.keys())[0])
    print(data[list(data.keys())[0]])
    print(list(data.keys())[-1])
    print(data[list(data.keys())[-1]])
    print(len(data))

data/raw/factify/extracted/images/test/0_evidence.jpg
tensor([0.0000, 0.0000, 0.0750,  ..., 0.2198, 0.0000, 0.0027])
data/raw/factify/extracted/images/train/9_evidence.jpg
tensor([0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0383, 0.0000])
41256
